In [112]:
import numpy as np
import pandas as pd
import random
import copy

In [233]:
class ChessBoard:
    def __init__(self):
        self.letters = 'abcdefgh'
        self.letNum = {'a': 1,
                      'b': 2,
                      'c': 3,
                      'd': 4,
                      'e': 5,
                      'f': 6,
                      'g': 7,
                      'h': 8}
        self.numLet = {v: k for k,v in self.letNum.items()}
        self.numbers = np.arange(1,9)
        self.board = {}
        self.board_prior = {}
        self.pieceSymbol = {'empty': ' ',
                     'pawn': 'P',
                     'bishop': 'B',
                     'knight': 'N',
                     'rook': 'R',
                     'queen': 'Q',
                     'king': 'K'}
        self.pieceID = {'empty': 0,
                     'pawn': 1,
                     'bishop': 2,
                     'knight': 3,
                     'rook': 4,
                     'queen': 5,
                     'king': 6}
        self.pieceName = {v: k for k,v in self.pieceID.items()}
        self.pieceValues = {0: 0,
                           1: 1,
                           2: 3,
                           3: 3,
                           4: 5,
                           5: 9,
                           6: 10}
        self.valuesToPID = {v: k for k,v in self.pieceValues.items()}
        self.side_to_move = 1
        self.in_check = False
        self.turn_number = 0
        self.board_threat_and_supports = {}
        self.tmp_bts = {}
        self.terminal_row_order = ['rook','knight','bishop','queen','king','bishop','knight','rook']
        self.game_start()
#         self.terminal_row_support = [0, 1, 1, 1, 1, 1, 1, 0]
#         self.pawn_row_support = [1, 1, 1, 4, 4, 1, 1, 1]
#         self.after_pawn_row_support = [1, 2, 3, 2, 2, 3, 2, 1]
        
    def game_start(self):
        self.board.update({f'{l}1': self.pieceID[self.terminal_row_order[p]] for p,l in enumerate(self.letters)})
        self.board.update({f'{l}2': self.pieceID['pawn'] for l in self.letters})
        self.board.update({f'{l}{n}': self.pieceID['empty'] for l in self.letters for n in self.numbers[2:6]})
        self.board.update({f'{l}7': self.pieceID['pawn']*-1 for l in self.letters})
        self.board.update({f'{l}8': self.pieceID[self.terminal_row_order[p]]*-1 for p,l in enumerate(self.letters)})
        self.board_prior = copy.deepcopy(self.board)
        
#         self.board_threat_and_supports.update({f'{l}1': self.terminal_row_support[p] for p,l in enumerate(self.letters)})
#         self.board_threat_and_supports.update({f'{l}2': self.pawn_row_support[p] for p,l in enumerate(self.letters)})
#         self.board_threat_and_supports.update({f'{l}3': self.after_pawn_row_support[p] for p,l in enumerate(self.letters)})
#         self.board_threat_and_supports.update({f'{l}{n}': 0 for l in self.letters for n in [4,5]})
#         self.board_threat_and_supports.update({f'{l}4': self.after_pawn_row_support[p]*-1 for p,l in enumerate(self.letters)})
#         self.board_threat_and_supports.update({f'{l}7': self.pawn_row_support[p]*-1 for p,l in enumerate(self.letters)})
#         self.board_threat_and_supports.update({f'{l}8': self.terminal_row_support[p]*-1 for p,l in enumerate(self.letters)})
        self.board_threat_and_supports.update({f'{l}{n}': 0 for l in self.letters for n in self.numbers})

    def visualize_board(self):
        print(f'Turn Number: {self.turn_number}')
        print("    ---------------------------------")
        for n in reversed(self.numbers):
            print(f"{n}   "
                 f"| {self.pieceSymbol[self.pieceName[abs(self.board[f'a{n}'])]]} |"\
                 f" {self.pieceSymbol[self.pieceName[abs(self.board[f'b{n}'])]]} |"\
                 f" {self.pieceSymbol[self.pieceName[abs(self.board[f'c{n}'])]]} |"\
                 f" {self.pieceSymbol[self.pieceName[abs(self.board[f'd{n}'])]]} |"\
                 f" {self.pieceSymbol[self.pieceName[abs(self.board[f'e{n}'])]]} |"\
                 f" {self.pieceSymbol[self.pieceName[abs(self.board[f'f{n}'])]]} |"\
                 f" {self.pieceSymbol[self.pieceName[abs(self.board[f'g{n}'])]]} |"\
                 f" {self.pieceSymbol[self.pieceName[abs(self.board[f'h{n}'])]]} |")
            print("    ---------------------------------")
        print("                                     ")
        print("      a   b   c   d   e   f   g   h")
        
    def visualize_board_prior(self):
        print(f'Turn Number: {self.turn_number-1}')
        print("    ---------------------------------")
        for n in reversed(self.numbers):
            print(f"{n}   "
                 f"| {self.pieceSymbol[self.pieceName[abs(self.board_prior[f'a{n}'])]]} |"\
                 f" {self.pieceSymbol[self.pieceName[abs(self.board_prior[f'b{n}'])]]} |"\
                 f" {self.pieceSymbol[self.pieceName[abs(self.board_prior[f'c{n}'])]]} |"\
                 f" {self.pieceSymbol[self.pieceName[abs(self.board_prior[f'd{n}'])]]} |"\
                 f" {self.pieceSymbol[self.pieceName[abs(self.board_prior[f'e{n}'])]]} |"\
                 f" {self.pieceSymbol[self.pieceName[abs(self.board_prior[f'f{n}'])]]} |"\
                 f" {self.pieceSymbol[self.pieceName[abs(self.board_prior[f'g{n}'])]]} |"\
                 f" {self.pieceSymbol[self.pieceName[abs(self.board_prior[f'h{n}'])]]} |")
            print("    ---------------------------------")
        print("                                     ")
        print("      a   b   c   d   e   f   g   h")
    
    def pawn_moves(self,sq,c,board,board_threat_and_supports):
        l = sq[0]
        n = int(sq[1])
        moves = []
        # Move forward 1
        if board == self.board:
            bp = self.board_prior
        else:
            bp = self.board
        if board[f'{l}{n+c}'] == 0:
            moves.append(f'{l}{n+c}')
            # Move forward 2
            if board[f'{l}{n+2*c}'] == 0 and (n*c == 2 or n*c == -7) and board[sq] == bp[sq]:
                moves.append(f'{l}{n+2*c}')
        
        l_n = self.letNum[l]
        # Capture left diagonal
        if (1 <= l_n-c <=8):
            board_threat_and_supports[f'{self.numLet[l_n-c]}{n+c}'] += board[f'{self.numLet[l_n-c]}{n+c}']*c
            if board[f'{self.numLet[l_n-c]}{n+c}']*c < 0:
                moves.append(f'{self.numLet[l_n-c]}{n+c}')
            
        # Capture right diagonal
        if (1 <= l_n+c <=8):
            board_threat_and_supports[f'{self.numLet[l_n+c]}{n+c}'] += board[f'{self.numLet[l_n+c]}{n+c}']*c
            if board[f'{self.numLet[l_n+c]}{n+c}']*c < 0:
                moves.append(f'{self.numLet[l_n+c]}{n+c}')

        # En Passant (add later)
        if moves:    
            return moves
    
    def bishop_moves(self,sq,c,board,board_threat_and_supports):
        l = sq[0]
        n = int(sq[1])
        l_n = self.letNum[l]
        moves = []
        # Upper right
        temp_ln = l_n + c
        temp_n = n + c
        while (1 <= temp_ln <=8) and (1 <= temp_n <=8):
            board_threat_and_supports[f'{self.numLet[temp_ln]}{temp_n}'] += board[f'{self.numLet[temp_ln]}{temp_n}']*c
            if board[f'{self.numLet[temp_ln]}{temp_n}'] == 0:
                moves.append(f'{self.numLet[temp_ln]}{temp_n}')
            elif board[f'{self.numLet[temp_ln]}{temp_n}']*c < 0:
                moves.append(f'{self.numLet[temp_ln]}{temp_n}')
                break
            elif board[f'{self.numLet[temp_ln]}{temp_n}']*c > 0:
                break
            temp_ln += c
            temp_n += c
        # Upper left
        temp_ln = l_n - c
        temp_n = n + c
        while (1 <= temp_ln <=8) and (1 <= temp_n <=8):
            board_threat_and_supports[f'{self.numLet[temp_ln]}{temp_n}'] += board[f'{self.numLet[temp_ln]}{temp_n}']*c
            if board[f'{self.numLet[temp_ln]}{temp_n}'] == 0:
                moves.append(f'{self.numLet[temp_ln]}{temp_n}')
            elif board[f'{self.numLet[temp_ln]}{temp_n}']*c < 0:
                moves.append(f'{self.numLet[temp_ln]}{temp_n}')
                break
            elif board[f'{self.numLet[temp_ln]}{temp_n}']*c > 0:
                break
            temp_ln -= c
            temp_n += c
        # Lower right
        temp_ln = l_n + c
        temp_n = n - c
        while (1 <= temp_ln <=8) and (1 <= temp_n <=8):
            board_threat_and_supports[f'{self.numLet[temp_ln]}{temp_n}'] += board[f'{self.numLet[temp_ln]}{temp_n}']*c
            if board[f'{self.numLet[temp_ln]}{temp_n}'] == 0:
                moves.append(f'{self.numLet[temp_ln]}{temp_n}')
            elif board[f'{self.numLet[temp_ln]}{temp_n}']*c < 0:
                moves.append(f'{self.numLet[temp_ln]}{temp_n}')
                break
            elif board[f'{self.numLet[temp_ln]}{temp_n}']*c > 0:
                break
            temp_ln += c
            temp_n -= c
        # Lower left
        temp_ln = l_n - c
        temp_n = n - c
        while (1 <= temp_ln <=8) and (1 <= temp_n <=8):
            board_threat_and_supports[f'{self.numLet[temp_ln]}{temp_n}'] += board[f'{self.numLet[temp_ln]}{temp_n}']*c
            if board[f'{self.numLet[temp_ln]}{temp_n}'] == 0:
                moves.append(f'{self.numLet[temp_ln]}{temp_n}')
            elif board[f'{self.numLet[temp_ln]}{temp_n}']*c < 0:
                moves.append(f'{self.numLet[temp_ln]}{temp_n}')
                break
            elif board[f'{self.numLet[temp_ln]}{temp_n}']*c > 0:
                break
            temp_ln -= c
            temp_n -= c
        if moves:
            return moves
    
    def knight_moves(self,sq,c,board,board_threat_and_supports):
        l = sq[0]
        n = int(sq[1])
        l_n = self.letNum[l]
        moves = []
        # Up/Down, right/Left
        for i in [1,-1]:
            for j in [2,-2]:
                if (1 <= l_n+i <=8) and (1 <= n+j <=8):
                    board_threat_and_supports[f'{self.numLet[l_n+i]}{n+j}'] += board[f'{self.numLet[l_n+i]}{n+j}']*c
                    if board[f'{self.numLet[l_n+i]}{n+j}']*c <= 0:
                        moves.append(f'{self.numLet[l_n+i]}{n+j}')
        # Right/Left, up,down
        for i in [2,-2]:
            for j in [1,-1]:
                if (1 <= l_n+i <=8) and (1 <= n+j <=8):
                    board_threat_and_supports[f'{self.numLet[l_n+i]}{n+j}'] += board[f'{self.numLet[l_n+i]}{n+j}']*c
                    if board[f'{self.numLet[l_n+i]}{n+j}']*c <= 0:
                        moves.append(f'{self.numLet[l_n+i]}{n+j}')
        if moves:
            return moves
    
    def rook_moves(self,sq,c,board,board_threat_and_supports):
        l = sq[0]
        n = int(sq[1])
        l_n = self.letNum[l]
        moves = []
        # Right
        temp_ln = l_n + c
        while (1 <= temp_ln <=8):
            board_threat_and_supports[f'{self.numLet[temp_ln]}{n}'] += board[f'{self.numLet[temp_ln]}{n}']*c
            if board[f'{self.numLet[temp_ln]}{n}'] == 0:
                moves.append(f'{self.numLet[temp_ln]}{n}')
            elif board[f'{self.numLet[temp_ln]}{n}']*c < 0:
                moves.append(f'{self.numLet[temp_ln]}{n}')
                break
            elif board[f'{self.numLet[temp_ln]}{n}']*c > 0:
                break
            temp_ln += c
        # Left
        temp_ln = l_n - c
        while (1 <= temp_ln <=8):
            board_threat_and_supports[f'{self.numLet[temp_ln]}{n}'] += board[f'{self.numLet[temp_ln]}{n}']*c
            if board[f'{self.numLet[temp_ln]}{n}'] == 0:
                moves.append(f'{self.numLet[temp_ln]}{n}')
            elif board[f'{self.numLet[temp_ln]}{n}']*c < 0:
                moves.append(f'{self.numLet[temp_ln]}{n}')
                break
            elif board[f'{self.numLet[temp_ln]}{n}']*c > 0:
                break
            temp_ln -= c
        # Up
        temp_n = n + c
        while (1 <= temp_n <=8):
            board_threat_and_supports[f'{self.numLet[l_n]}{temp_n}'] += board[f'{self.numLet[l_n]}{temp_n}']*c
            if board[f'{self.numLet[l_n]}{temp_n}'] == 0:
                moves.append(f'{self.numLet[l_n]}{temp_n}')
            elif board[f'{self.numLet[l_n]}{temp_n}']*c < 0:
                moves.append(f'{self.numLet[l_n]}{temp_n}')
                break
            elif board[f'{self.numLet[l_n]}{temp_n}']*c > 0:
                break
            temp_n += c
        # Down
        temp_n = n - c
        while (1 <= temp_n <=8):
            board_threat_and_supports[f'{self.numLet[l_n]}{temp_n}'] += board[f'{self.numLet[l_n]}{temp_n}']*c
            if board[f'{self.numLet[l_n]}{temp_n}'] == 0:
                moves.append(f'{self.numLet[l_n]}{temp_n}')
            elif board[f'{self.numLet[l_n]}{temp_n}']*c < 0:
                moves.append(f'{self.numLet[l_n]}{temp_n}')
                break
            elif board[f'{self.numLet[l_n]}{temp_n}']*c > 0:
                break
            temp_n -= c
        if moves:
            return moves
    
    def queen_moves(self,sq,c,board,board_threat_and_supports):
        moves = []
        b_moves = self.bishop_moves(sq,c,board,board_threat_and_supports)
        if b_moves:
            moves.append(b_moves)
        r_moves = self.rook_moves(sq,c,board,board_threat_and_supports)
        if r_moves:
            moves.append(r_moves)
        moves_flat = [move for movelist in moves for move in movelist]
        if moves_flat:
            return moves_flat
    
    def king_moves(self,sq,c,board,board_threat_and_supports):
        l = sq[0]
        n = int(sq[1])
        l_n = self.letNum[l]
        moves = []
        for i in [1,0,-1]:
            for j in [1,0,-1]:
                if (1 <= l_n+i <=8) and (1 <= n+j <=8):
                    board_threat_and_supports[f'{self.numLet[l_n+i]}{n+j}'] += board[f'{self.numLet[l_n+i]}{n+j}']*c
                    if board[f'{self.numLet[l_n+i]}{n+j}']*c <= 0:
                        moves.append(f'{self.numLet[l_n+i]}{n+j}')
        if moves:
            return moves
    
    def gen_pseudolegal_moves(self,c,board,board_threat_and_supports):     
        pieces_on_board = []
        for sq,pID in board.items():
            if pID*c>=0:
                pieces_on_board.append([sq,pID])

        possible_moves = {}
        for sq,pID in pieces_on_board:
            # Pawn Moves
            if pID*c == 1:
                possible_moves[sq] = self.pawn_moves(sq,c,board,board_threat_and_supports)
            elif pID*c == 2:
                possible_moves[sq] = self.bishop_moves(sq,c,board,board_threat_and_supports)
            elif pID*c == 3:
                possible_moves[sq] = self.knight_moves(sq,c,board,board_threat_and_supports)
            elif pID*c == 4:
                possible_moves[sq] = self.rook_moves(sq,c,board,board_threat_and_supports)
            elif pID*c == 5:
                possible_moves[sq] = self.queen_moves(sq,c,board,board_threat_and_supports)
            elif pID*c == 6:
                possible_moves[sq] = self.king_moves(sq,c,board,board_threat_and_supports)
        possible_moves = {k: v for k,v in possible_moves.items() if v is not None}

        return possible_moves
    
    def is_King_in_check(self,c,board):
        rev_board = {v: k for k,v in board.items()}
        king_pos = rev_board[6*c]
        opponent_moves = self.gen_pseudolegal_moves(-c,board,self.tmp_bts)
        in_check = False
        for k,vlist in opponent_moves.items():
            if king_pos in vlist:
                in_check = True
                break
        return in_check
        
    
    def check_legality(self,c,possible_moves,board):        
        legal_moves = {}
        # Gen list of all opponent piece
        lm_bts_vals = {}
        for piece,moves in possible_moves.items():
            lms_per_piece = []
            summed_bts_vals = []
            for move in moves:
                tmp_board = copy.deepcopy(board)
                tmp_board.update({f'{move}': board[f'{piece}']})
                tmp_board.update({f'{piece}': 0})
                self.tmp_bts = copy.deepcopy(self.board_threat_and_supports)
                if not self.is_King_in_check(self.side_to_move,tmp_board):
                    lms_per_piece.append(move)
                    bts_diff = {k:(self.board_threat_and_supports[k] - self.tmp_bts[k])*-c for k in moves}
                    #print(f'piece: {piece}\nmove: {move}\nself.board_threat_and_supports: {self.board_threat_and_supports}\ntmp_board_threat_and_supports: {tmp_board_threat_and_supports}\nbts_diff: {bts_diff}\n')
                    summed_bts_vals.append(bts_diff[move])
                    if piece == 'f3' and move == 'f6':
                        print(bts_diff[move])
                        #print(f'board_threat_and_supports[key]: {board_threat_and_supports[move]}\nself.tmp_bts[key]: {self.tmp_bts[move]}')
                        #print(f'piece: {piece}\nmove: {move}\nself.bts: {self.board_threat_and_supports[piece]}\nself.tmp_bts: {self.tmp_bts[piece]}')
            if lms_per_piece:
                legal_moves[f'{piece}'] = lms_per_piece
                #print(f'tmp_board_threat_and_supports: {tmp_board_threat_and_supports}')
                #print(f'summed_bts_vals: {summed_bts_vals}')
                lm_bts_vals[f'{piece}'] = summed_bts_vals
        #print(f'CHECK_LEGALITY   lm: {legal_moves}\n lm_bts_vals: {lm_bts_vals}\n')
        
        return legal_moves,lm_bts_vals
        
    def picked_move(self,picked_key,legal_moves,lm_vals):
        if self.side_to_move == 1:
            color = 'White'
        elif self.side_to_move == -1:
            color = 'Black'
        max_val = max(lm_vals[picked_key])
        max_val_ind = lm_vals[picked_key].index(max_val)
        move = legal_moves[picked_key][max_val_ind]
        #print(f'max_val: {max_val}\npiece_start: {picked_key}\npiece_end: {move}\n')
        print(f'{color} plays {self.pieceSymbol[self.pieceName[abs(self.board[picked_key])]]}{picked_key} to {move}')
        self.board_prior = copy.deepcopy(self.board)
        self.board.update({f'{move}': self.board[f'{picked_key}']})
        self.board.update({f'{picked_key}': 0})
        self.side_to_move = -self.side_to_move
        self.turn_number += 1
        self.board_threat_and_supports.update({k: 0 for k in self.board_threat_and_supports.keys()})
        
    def play(self,num_moves):
        self.visualize_board()
        for _ in range(num_moves):
            pm = self.gen_pseudolegal_moves(self.side_to_move,self.board,self.board_threat_and_supports)
            lm,lm_bts_vals = self.check_legality(self.side_to_move,pm,self.board)
            print(f'play_lm_bts_vals: {lm_bts_vals}\n')
            #print(f'lm: {lm}\nlm_bts_vals: {lm_bts_vals}\n')
            lm_vals_capture = self.value_from_capture(lm,self.board)
            lm_vals_Ocontrol = self.value_from_offense_control(lm,self.board,self.board_threat_and_supports)            
            lm_vals = {k: [float(lm_vals_capture[k][i]) + float(lm_vals_Ocontrol[k][i]) + float(lm_bts_vals[k][i]) for i in range(len(lm_vals_capture[k]))] for k in lm_vals_capture.keys()}                        
            #print(f'lm_vals: {lm_vals}\nlm_bts_vals: {lm_bts_vals}\n')
            max_val_piece = max(lm_vals, key= lambda k: max(lm_vals[k]))
            self.picked_move(max_val_piece,lm,lm_vals)
            self.visualize_board()
            print("\n\n\n\n\n")

    def value_from_capture(self,legal_moves,board):
        
        lm_vals = copy.deepcopy(legal_moves)
        
        for piece,moves in legal_moves.items():
            piece_moves = []
            for move in moves:
                piece_moves.append(self.pieceValues[board[move]*-self.side_to_move])
            lm_vals[piece] = piece_moves
        return lm_vals
    
    def value_from_offense_control(self,legal_moves,board,board_threat_and_supports):
        
        lm_vals = copy.deepcopy(legal_moves)
        bts_vals = {}
        for piece,moves in legal_moves.items():
            summed_move_vals = []
            summed_bts_vals = []
            for move in moves:
                move_vals = []
                tmp_board = copy.deepcopy(board)
                tmp_board.update({f'{move}': board[f'{piece}']})
                tmp_board.update({f'{piece}': 0})
                tmp_board_bts = copy.deepcopy(board_threat_and_supports)
                pm_from_move = self.gen_pseudolegal_moves(self.side_to_move,tmp_board,tmp_board_bts)
                lm_from_move,lm_bts_vals = self.check_legality(self.side_to_move,pm_from_move,tmp_board)                                
                #print(f'Ocontrol lm_bts" {lm_bts_vals}')
                for p,mset in lm_from_move.items():                    
                    move_val = 0
                    for m in mset:
                        if tmp_board[m] == 0:
                            move_val += 0.05
                        elif tmp_board[m]*self.side_to_move > 0:
                            move_val += 0.2*self.pieceValues[tmp_board[m]*self.side_to_move]
                        elif tmp_board[m]*self.side_to_move < 0:                           
                            move_val += 0.1*self.pieceValues[tmp_board[m]*-self.side_to_move]
                    move_vals.append(move_val)
                #print(f'lm_bts_vals: {lm_bts_vals.values()}')
                #summed_bts_vals.append(sum([int(v) for v in lm_bts_vals.values()]))
                summed_move_vals.append(sum(move_vals))  
            lm_vals[piece] = summed_move_vals
            #bts_vals[piece] = summed_bts_vals
        return lm_vals#, bts_vals
    

Next things to add:
 - Determine if opponent is in check
 - Determine checkmate
 - Determine stalemate
 - Minimax algorithm for choosing moves
 - Castling
 - En passant
 
Position evaluation ideas:
 - Basic point tracking for pieces
 - Minor points (0.1?) for controlling empty squares
 - Minor points (0.2?) for threatening enemy pieces (with allied support?)
 - Minor points (0.25?) for supporting allied pieces
 - Minor points for pins, forks, etc.
 
Remember:
 - It is the exception that a move only changes it's own val_from_Ocontrol, not the rule. val_from_Ocontrol needs to assess all of its own pieces.
 - To give value to pins is it necessary to do val_from_Ocontrol from the opposite side's perspective? The thinking being that pins take away opponent opportunities

In [234]:
game1 = ChessBoard()


In [235]:
game1.play(20)

Turn Number: 0
    ---------------------------------
8   | R | N | B | Q | K | B | N | R |
    ---------------------------------
7   | P | P | P | P | P | P | P | P |
    ---------------------------------
6   |   |   |   |   |   |   |   |   |
    ---------------------------------
5   |   |   |   |   |   |   |   |   |
    ---------------------------------
4   |   |   |   |   |   |   |   |   |
    ---------------------------------
3   |   |   |   |   |   |   |   |   |
    ---------------------------------
2   | P | P | P | P | P | P | P | P |
    ---------------------------------
1   | R | N | B | Q | K | B | N | R |
    ---------------------------------
                                     
      a   b   c   d   e   f   g   h
play_lm_bts_vals: {'b1': [0, 0], 'g1': [0, 0], 'a2': [0, 0], 'b2': [0, 0], 'c2': [0, 0], 'd2': [0, 0], 'e2': [0, 0], 'f2': [0, 0], 'g2': [0, 0], 'h2': [0, 0]}

White plays Pe2 to e3
Turn Number: 1
    ---------------------------------
8   | R | N | B | Q | K | B | 

White plays Qb7 to c8
Turn Number: 9
    ---------------------------------
8   | R |   | Q |   | K | B | N | R |
    ---------------------------------
7   | P |   | P | P | P | P | P | P |
    ---------------------------------
6   | P |   |   |   |   |   |   |   |
    ---------------------------------
5   |   |   |   |   |   |   |   |   |
    ---------------------------------
4   |   |   |   |   |   |   |   |   |
    ---------------------------------
3   |   |   |   |   | P |   |   |   |
    ---------------------------------
2   | P | P | P | P |   | P | P | P |
    ---------------------------------
1   | R | N | B |   | K |   | N | R |
    ---------------------------------
                                     
      a   b   c   d   e   f   g   h






play_lm_bts_vals: {'a8': [0]}

Black plays Ra8 to c8
Turn Number: 10
    ---------------------------------
8   |   |   | R |   | K | B | N | R |
    ---------------------------------
7   | P |   | P | P | P | P | P | P |
    ------------

White plays Pd2 to c3
Turn Number: 19
    ---------------------------------
8   |   |   | R |   | K | B |   | R |
    ---------------------------------
7   | P |   | P |   | P | P | P | P |
    ---------------------------------
6   | P |   |   |   |   |   |   |   |
    ---------------------------------
5   |   |   |   |   |   |   |   |   |
    ---------------------------------
4   |   |   |   |   | P |   |   |   |
    ---------------------------------
3   |   |   | P |   |   |   |   |   |
    ---------------------------------
2   | P | P | P |   | N | P | P | P |
    ---------------------------------
1   | R |   | B |   | K |   |   | R |
    ---------------------------------
                                     
      a   b   c   d   e   f   g   h






play_lm_bts_vals: {'a6': [0], 'c7': [0, 0], 'e7': [0, 0], 'f7': [0, 1], 'g7': [0, 1], 'h7': [1, 0], 'c8': [0, 0, 0], 'e8': [0, 0], 'h8': [0]}

Black plays Pf7 to f5
Turn Number: 20
    ---------------------------------
8   |   |   | R |